In [1]:
import pandas as pd
import json


In [2]:
with open("config.json", 'r') as f:
    args = json.load(f)
with open("MP_ASSET_STATE.csv","r") as outfile:
    MP_Asset_State = pd.read_csv(outfile, index_col=0)
with open("MP_ASSET.csv","r") as outfile:
    MP_Asset = pd.read_csv(outfile, index_col=0)

LS_Opening = pd.read_csv("LS_Opening",index_col=0)
LP_Deposit = pd.read_csv("LP_Deposit",index_col=0)
LP_Withdraw = pd.read_csv("LP_Withdraw",index_col=0)
SYS_LP_Withdraw = pd.read_csv("SYS_LP_Withdraw",index_col=0)


LS_Repayment = pd.read_csv("LS_Repayment",index_col=0)

LS_Closing = pd.read_csv("LS_Closing",index_col=0)
LS_Liquidation = pd.read_csv("LS_Liquidation",index_col=0)
LP_Pool_State = pd.read_csv("LP_Pool_State.csv",index_col=0)




/home/adept/anaconda3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
def LP_pool_gen(args):
    list_id =["pid" + str(sub) for sub in range(100,100+len(args["Pool_Assets"]))]
    #
    lpp = pd.DataFrame({"LP_Pool_id":list_id,"LP_symbol":args["Pool_Assets"]})
    return lpp
pool_id = LP_pool_gen(args)

In [4]:
def sum_by_day(LP_Pool_State,LS_Opening,MP_Asset,pool_id,args):#Check!!
    l=LS_Opening.groupby(["LS_timestamp","LP_Pool_id"]).sum()#groupby(['LP_Pool_id'])
    l = l.reset_index()
    ids = pool_id.loc[pool_id["LP_symbol"] == args["currency_stable"]]["LP_Pool_id"]
    ids.values[0]
    stable = l.loc[l["LP_Pool_id"]== ids.values[0]]
    stable.reset_index()
    asset = stable["LS_loan_amnt_stable"]
    asset = l.loc[l["LP_Pool_id"]!= ids.values[0]]
    asset = asset[["LS_timestamp","LS_loan_amnt_stable","LP_Pool_id"]]

    asset = pd.merge(asset,pool_id,on="LP_Pool_id", how='left')
    asset = asset.rename(columns={"LP_symbol":"symbol"})
    symbol_digit = pd.DataFrame(args["symbol_digit"])
    asset = pd.merge(asset,symbol_digit, on="symbol", how="left")
    m=MP_Asset
    m = m.rename(columns={"MP_asset_symbol":"symbol","MP_timestamp":"LS_timestamp"})

    asset = pd.merge(asset,m,on=["LS_timestamp","symbol"],how="left")
    asset["LS_loan_amnt_stable"] = asset["LS_loan_amnt_stable"]/(asset["MP_price_in_stable"]/10**asset["digit"])
    asset["LS_loan_amnt_stable"] =asset["LS_loan_amnt_stable"].round(0).astype("uint64")
    asset = asset.drop(["symbol","digit","MP_price_in_stable"], axis=1)
    #print(asset,stable)
    asset = asset.rename(columns={"LS_timestamp":"LP_Pool_timestamp","LS_loan_amnt_stable":"LP_Pool_total_borrowed_asset"})
    stable = stable[["LS_timestamp","LS_loan_amnt_stable","LP_Pool_id"]]
    stable = stable.rename(columns={"LS_timestamp":"LP_Pool_timestamp","LS_loan_amnt_stable":"LP_Pool_total_borrowed_stable"})

    LP_Pool_State = pd.merge(LP_Pool_State,stable,on=["LP_Pool_timestamp","LP_Pool_id"],how="left").fillna(0)
    LP_Pool_State = pd.merge(LP_Pool_State,asset,on=["LP_Pool_timestamp","LP_Pool_id"],how="left").fillna(0)
    return LP_Pool_State

In [5]:
def lp_tdep(LP_Pool_State,LP_Deposit,pool_id,args):
        l=LP_Deposit.groupby(["LP_timestamp","LP_Pool_id"]).sum()
        l=l.reset_index()
        ids = pool_id.loc[pool_id["LP_symbol"] == args["currency_stable"]]["LP_Pool_id"]
        asset = l.loc[l["LP_Pool_id"]!= ids.values[0]]
        asset.reset_index()
        asset = asset[["LP_timestamp","LP_amnt_asset","LP_Pool_id"]]
        stable = l[["LP_timestamp","LP_amnt_stable","LP_Pool_id",]]
        new = pd.merge(stable,asset,on=["LP_timestamp","LP_Pool_id"],how="left")
        new = new.rename(columns={"LP_timestamp":"LP_Pool_timestamp","LP_amnt_stable":"LP_Pool_total_deposited_stable","LP_amnt_asset":"LP_Pool_total_deposited_asset"})
        LP_Pool_State = pd.merge(LP_Pool_State,new,on=["LP_Pool_timestamp","LP_Pool_id"], how='left')
        return LP_Pool_State 

In [6]:
def LP_Pool_State_gen(MP_Asset,LS_Opening,LP_Pool,args):#LP_Deposit,LP_Withdraw,LP_Pool,args):
    timestamp = MP_Asset.drop_duplicates(subset=["MP_timestamp"], keep="last")
    
    LP_Pool_State = pd.DataFrame()
    for pool in pool_id["LP_Pool_id"]:
        tmp = pd.DataFrame({"LP_Pool_timestamp":timestamp["MP_timestamp"],"LP_Pool_id":pool})
        LP_Pool_State = pd.concat([LP_Pool_State,tmp],axis=0, ignore_index=True)
        
    LP_Pool_State = LP_Pool_State.sort_values("LP_Pool_timestamp",ignore_index=True)
    LP_Pool_State = sum_by_day(LP_Pool_State, LS_Opening, MP_Asset, pool_id, args)
    LP_Pool_State["LP_Pool_total_borrowed_stable"] =  LP_Pool_State["LP_Pool_total_borrowed_stable"] + LP_Pool_State["LP_Pool_total_borrowed_asset"]
    LP_Pool_State = lp_tdep(LP_Pool_State,LP_Deposit,pool_id,args).fillna(0)
    ids = pool_id.loc[pool_id["LP_symbol"] == args["currency_stable"]]["LP_Pool_id"]

    #LP_Pool_State.loc[LP_Pool_State["LP_Pool_id"] == ids.values[0], ['LP_Pool_total_borrowed_asset']] = LP_Pool_State["LP_Pool_total_borrowed_stable"].loc[LP_Pool_State["LP_Pool_id"] == ids.values[0]]
    #LP_Pool_State.loc[LP_Pool_State["LP_Pool_id"] == ids.values[0], ['LP_Pool_total_deposited_asset']] = LP_Pool_State["LP_Pool_total_deposited_stable"].loc[LP_Pool_State["LP_Pool_id"] == ids.values[0]]
    
    a =pd.DataFrame(args["symbol_digit"])
    c = a.loc[a["symbol"] == args["currency_stable"]]["digit"].values[0].astype(float)
    LP_Pool_State.loc[LP_Pool_State["LP_Pool_timestamp"] == min(LP_Pool_State["LP_Pool_timestamp"]),["LP_Pool_total_deposited_stable"]] =args["startup_pool_value"]*10**c
    LP_Pool_State["SYS_LS_interest"] = 0
    LP_Pool_State["SYS_LS_Pool_interest"] = 0
    LP_Pool_State["SYS_TR_interest"] = 0
    #LP_total_deposited is not yet deletet - probably will be removed in next version
    LP_Pool_State["SYS_LP_Pool_TV_IntDep_stable"] = LP_Pool_State["LP_Pool_total_deposited_stable"]
    
    #cumsum
    #LP_Pool_State["LP_Pool_total_borrowed_stable"] = LP_Pool_State[['LP_Pool_id', 'LP_Pool_total_borrowed_stable']].groupby('LP_Pool_id').cumsum()
    #LP_Pool_State["LP_Pool_total_borrowed_asset"] = LP_Pool_State[['LP_Pool_id', 'LP_Pool_total_borrowed_asset']].groupby('LP_Pool_id').cumsum()
    #LP_Pool_State["LP_Pool_total_deposited_stable"] = LP_Pool_State[['LP_Pool_id', 'LP_Pool_total_deposited_stable']].groupby('LP_Pool_id').cumsum()
    #LP_Pool_State["LP_Pool_total_deposited_asset"] = LP_Pool_State[['LP_Pool_id', 'LP_Pool_total_deposited_asset']].groupby('LP_Pool_id').cumsum()
    
    return LP_Pool_State

p = LP_Pool_State_gen(MP_Asset,LS_Opening,pool_id,args)
p.to_csv("LP_Pool_State.csv")
    

In [7]:
p

,LP_Pool_timestamp,LP_Pool_id,LP_Pool_total_borrowed_stable,LP_Pool_total_borrowed_asset,LP_Pool_total_deposited_stable,LP_Pool_total_deposited_asset,SYS_LS_interest,SYS_LS_Pool_interest,SYS_TR_interest,SYS_LP_Pool_TV_IntDep_stable
0,2020-08-21 03:00:00,pid100,0.000000e+00,0.000000e+00,1.000000e+18,0.000000e+00,0,0,0,1.000000e+18
1,2020-08-21 03:00:00,pid101,0.000000e+00,0.000000e+00,1.000000e+18,0.000000e+00,0,0,0,1.000000e+18
2,2020-08-22 03:00:00,pid101,1.736194e+16,1.736194e+16,0.000000e+00,0.000000e+00,0,0,0,0.000000e+00
3,2020-08-22 03:00:00,pid100,6.314495e+11,0.000000e+00,2.771500e+16,0.000000e+00,0,0,0,2.771500e+16
4,2020-08-23 03:00:00,pid100,6.137602e+16,0.000000e+00,1.631200e+16,0.000000e+00,0,0,0,1.631200e+16
...,...,...,...,...,...,...,...,...,...,...
1457,2022-08-19 03:00:00,pid100,7.704555e+17,0.000000e+00,1.460750e+17,0.000000e+00,0,0,0,1.460750e+17
1458,2022-08-20 03:00:00,pid101,3.924169e+17,3.924169e+17,8.344897e+16,8.300400e+16,0,0,0,8.344897e+16
1459,2022-08-20 03:00:00,pid100,7.738081e+17,0.000000e+00,1.264230e+17,0.000000e+00,0,0,0,1.264230e+17
1460,2022-08-21 03:00:00,pid100,3.775335e+17,0.000000e+00,1.254630e+17,0.000000e+00,0,0,0,1.254630e+17


In [13]:
p.loc[p["LP_Pool_timestamp"] == min(p["LP_Pool_timestamp"])]

,LP_Pool_timestamp,LP_Pool_id,LP_Pool_total_borrowed_stable,LP_Pool_total_borrowed_asset,LP_Pool_total_deposited_stable,LP_Pool_total_deposited_asset
0,2020-08-21 03:00:00,pid100,0.0,0.0,1.000000e+18,0.0
1,2020-08-21 03:00:00,pid101,0.0,0.0,1.000000e+18,0.0


In [19]:
p.to_csv("LP_Pool_State.csv")

In [21]:
p.loc[p["LP_Pool_id"]=="pid101"]

,LP_Pool_timestamp,LP_Pool_id,LP_Pool_total_borrowed_stable,LP_Pool_total_borrowed_asset,LP_Pool_total_deposited_stable,LP_Pool_total_deposited_asset
1,2020-08-21 03:00:00,pid101,0.000000e+00,0.000000e+00,1.000000e+18,0.000000e+00
2,2020-08-22 03:00:00,pid101,1.736194e+16,1.736194e+16,0.000000e+00,0.000000e+00
5,2020-08-23 03:00:00,pid101,3.361789e+16,3.361789e+16,1.015058e+16,1.015300e+16
7,2020-08-24 03:00:00,pid101,4.914529e+16,4.914529e+16,0.000000e+00,0.000000e+00
9,2020-08-25 03:00:00,pid101,8.334203e+16,8.334203e+16,3.322628e+16,3.321500e+16
...,...,...,...,...,...,...
1452,2022-08-17 03:00:00,pid101,4.773092e+17,4.773092e+17,4.215951e+15,4.207000e+15
1454,2022-08-18 03:00:00,pid101,4.816016e+17,4.816016e+17,2.158002e+16,2.155600e+16
1456,2022-08-19 03:00:00,pid101,4.861899e+17,4.861899e+17,1.111487e+16,1.109800e+16
1458,2022-08-20 03:00:00,pid101,3.924169e+17,3.924169e+17,1.157472e+16,1.151300e+16
